# Análise Exploratória de Dados (EDA) - Jogos Olímpicos
**Grupo:** G03 | **Unidade Curricular:** IA25

## Objetivos
1. Análise demográfica e temporal dos atletas.
2. Correlação entre atributos físicos e sucesso (medalhas).
3. **Feature Engineering:** Preparação de dados para regressão (Previsão de Medalhas por País).

In [ ]:
# Importação de bibliotecas essenciais
%pip install pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração estética dos gráficos
sns.set(style="whitegrid", context="notebook")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Carregamento do dataset
try:
    df = pd.read_csv("athlete_events.csv")
    print(f"Dataset carregado com sucesso! Dimensões: {df.shape}")
except FileNotFoundError:
    print("ERRO: O ficheiro 'athlete_events.csv' não foi encontrado.")

# Visualização inicial
display(df.head())

## 1. Feature Engineering: Décadas
Para facilitar a visualização temporal e reduzir o ruído no eixo X, agrupamos os anos em décadas.

In [ ]:
# Criar coluna 'Decade'
df['Decade'] = (df['Year'] // 10) * 10

# Visualizar contagem de atletas por Década
plt.figure(figsize=(12, 6))
sns.countplot(x="Decade", data=df, palette="viridis")
plt.title("Evolução do Número de Atletas por Década")
plt.xlabel("Década")
plt.ylabel("Número de Atletas")
plt.xticks(rotation=45)
plt.show()

## 2. Distribuição de Atributos Físicos
Análise da Idade, Altura e Peso.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(df['Age'].dropna(), bins=30, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Distribuição de Idade')

sns.histplot(df['Height'].dropna(), bins=30, kde=True, ax=axes[1], color='salmon')
axes[1].set_title('Distribuição de Altura')

sns.histplot(df['Weight'].dropna(), bins=30, kde=True, ax=axes[2], color='green')
axes[2].set_title('Distribuição de Peso')

plt.tight_layout()
plt.show()

## 3. Relação Atributos Físicos vs. Medalhas
Os medalhistas têm atributos físicos diferentes da média? (Nota: Removemos colunas não numéricas para evitar erros).

In [ ]:
# Selecionar colunas numéricas relevantes
numeric_cols = ['Age', 'Height', 'Weight']

plt.figure(figsize=(15, 5))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(x="Medal", y=col, data=df, order=['Gold', 'Silver', 'Bronze'], palette="Set2")
    plt.title(f"{col} por Tipo de Medalha")

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de Correlação (apenas numéricos)
corr = df.select_dtypes(include=[np.number]).corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Matriz de Correlação")
plt.show()

In [ ]:
# Análise de Clusters: O Desporto define o corpo?
selected_sports = ['Basketball', 'Gymnastics', 'Weightlifting', 'Athletics']
sport_subset = df[df['Sport'].isin(selected_sports)].dropna(subset=['Height', 'Weight'])

plt.figure(figsize=(10, 6))
sns.scatterplot(data=sport_subset, x='Height', y='Weight', hue='Sport', alpha=0.6)
plt.title('Relação Altura vs Peso por Desporto (Labels Naturais)')
plt.show()

## 4. Preparação para Machine Learning (Regressão)
**Ideia:** Em vez de classificar se um atleta ganha ou não, vamos prever **quantas medalhas um país ganha por ano**.

Criamos um novo dataset agregado: `NOC` (País) | `Year` | `Medal_Count`.

In [ ]:
# Filtrar apenas linhas com medalhas
medals_only = df.dropna(subset=['Medal'])

# Agrupar por País, Ano e Década
country_performance = medals_only.groupby(['NOC', 'Year', 'Decade'])['Medal'].count().reset_index()
country_performance.rename(columns={'Medal': 'Medal_Count'}, inplace=True)

# Exibir o dataset transformado
print("Dataset preparado para Regressão (Top 5 linhas):")
display(country_performance.head())

# Visualizar os 5 países com mais medalhas históricas
top_countries = country_performance.groupby('NOC')['Medal_Count'].sum().nlargest(5).index
subset_top = country_performance[country_performance['NOC'].isin(top_countries)]

plt.figure(figsize=(12, 6))
sns.lineplot(data=subset_top, x='Decade', y='Medal_Count', hue='NOC', marker='o', ci=None)
plt.title("Evolução de Medalhas por Década (Top 5 Países)")
plt.ylabel("Total de Medalhas")
plt.show()

### Conclusão
Este dataset preparado (`country_performance`) pode agora ser usado para treinar modelos de Regressão Linear ou Random Forest Regressors, usando features como o histórico de medalhas, tamanho da delegação (que pode ser calculado do dataset original) e década.